## Technical Tutor Notebook Overview
- Loads `.env` configuration, initializes an OpenAI client (`gpt-4o-mini`), and prepares a reusable `timestamp` helper.
- Fetches reference material from `https://realpython.com/async-io-python/` via `fetch_website_contents`, trimming to stay within token limits.
- Maintains short-term conversation memory (`ConversationMemory`) to keep context across multiple tutoring questions.
- Defines `ask_tutor()` to build prompts from the question, conversation history, and scraped notes, then streams replies with `IPython.display`.
- Demonstrates the tutor responding to example questions about Python `asyncio`, ending each answer with “Need another example?”

In [8]:

import os
import sys
import textwrap
from datetime import datetime

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown,display,update_display

repo_root = "/Users/karthikpk/development/llm_engineering"
if repo_root not in sys.path:
    sys.path.insert(0,repo_root)
from week1.scraper import fetch_website_contents





In [17]:
load_dotenv(override=True)
openai_api_key= os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

client = OpenAI()
MODEL = "gpt-4o-mini"

timestamp = lambda: datetime.now().strftime("%H:%M:%S")

source_url = {
    "async-guide": "https://realpython.com/async-io-python/",
}



In [ ]:


def get_source_notes(url:str,max_chars:int = 2000) -> str:
    print(f"[{timestamp()}] Fetching {url}...")
    try:
        text=fetch_website_contents(url)
    except Exception as exec:
        text= f"(Failed to fetch {url}: {exec})"
    return text[:max_chars]


def build_context() -> str:
    snippets= []
    for name,url in  source_url.items():
        raw = get_source_notes(url)
        snippets.append(f"### {name}\nSource: {url}\n\n{raw}")
    combined = "\n\n".join(snippets)
    return combined[:4000]

session_context = build_context()





In [21]:
class ConversationMemory:
    def __init__(self,max_entries: int = 5, max_chars: int = 2000):
        self.entries = []
        self.max_entries = max_entries
        self.max_chars = max_chars

    def add_entry(self,question: str, answer:str)-> None:
        self.entries.append({"question": question.strip(), "answer": answer.strip()})
        if len(self.entries)> self.max_entries:
            self.entries=self.entries[-self.max_entries:]
    
    def build_context(self) -> str:
        chunks = []
        running  = 0
        for item in reversed(self.entries):
            block = textwrap.dedent(f"""
            Q: {item['question']}
            A:{item['answer']}
            """).strip()
            if running + len(block) > self.max_chars:
                break
            chunks.append(block)
            running+=len(block)
        return "\n\n".join(reversed(chunks))
 

In [20]:
memory = ConversationMemory(max_entries=5,max_chars=2000)

In [22]:
# --- Tutor answer (with streaming)
tutor_system_prompt = textwrap.dedent("""
    You are a friendly technical tutor.
    Use the provided context to explain concepts clearly.
    Prefer short paragraphs and include a quick tip or suggestion.
    End your reply with: "Need another example?"
""").strip()


def ask_tutor(question: str) -> None:
    past_context=memory.build_context()
    user_prompt = textwrap.dedent(f"""
    Question:
     {question}
    Recent conversation(most recent last):
    {past_context or "No previous conversation yet."}
    Reference notes:
    {session_context}
    """
    ).strip()

    stream = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": tutor_system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        stream=True,
    )


    reply, handle = "" , display(Markdown(""),display_id=True)
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        reply+=delta
        update_display(Markdown(reply),display_id=handle.display_id)
    memory.add_entry(question,reply)

In [ ]:
question = "Whats asyncio?"
ask_tutor(question)

In [ ]:
question = "Whats my first question?"
ask_tutor(question)

In [ ]:
question = "Whats my second question?"
ask_tutor(question)